# Non-Profits & Tech
> "Leveling the playing field"

- toc: true
- branch: master
- categories: [data, non-profits, civic tech, unnecessary visualizations]
- image: images/posts/datajawn/bees.jpg
- badges: false
- hide: true

In [8]:
#hide
import pandas as pd
import numpy as np
import altair as alt
from siuba import _, group_by, summarize, arrange

In the tech world, the folks programming and building the actual product generally don't think about costs (outside of developer time). For companies who make money from the product the technologists are building, it's really a matter of revenue.  A thriving charity's first success is being able to afford basic operations.  A non-proft has "made it" when they can say "what else can we pay for". For non-profits and government agencies, there's not a lot of passive (legal) revenue to the organization, so we're gonna cover that choose-your-own-adventure style.

## Making money the hard way 
### A primer for folks who haven't hustled to avoid revenue

Let's start with the assumption that all non-profits need some amount of funding. The most common methods for getting these funds are:
1. Donations 
2. Grants
3. Service Provider

#### Donations
For the sake of not letting myself ramble, we're going to focus on financial donations. Many non-profits exist on the model of donations of goods (like Goodwill), selling those goods, and then generating operating costs from those goods. It's a financial model.  It exists. The end.

Non-profit organizations normally accept financial donations, but simply relying on someone to wake up and think "you know what, I'm going to send a check to United Way today" is probably not going to result in a reliable stream of income, if any income at all.  If a non-profit wants a reliable amount of funds from donations, they need a fundraising campaign.  Fundraising campaigns are not free - if Susan G. Komen spends $200,000 buying fancy pink shoes for football players and raises $ 800,000 (in an imaginary land in which you can directly attribute all financial donations to a single campaign), they're actually raising $600,000.  They spent 25 cents for every dollar raised.  In order to include this money in an annual budget, the non-profit needs to hold regular fundraising campaigns.  This is why most non-profits will employ a Fundraising Coordinator or Marketing Specialist.  If they plan on relying on donations as a revenue stream, they need to efficiently and consistently fundraise. 

#### Grants
Grants are a sum of money given to an organization for a specific purpose.  Grants come in two basic flavors: a sweet but tangy public grant or an earthy, umami private grant.  In my experience, private grants are a little less stringent with their requirements but here's a basic breakdown of how to receive and use grant funding.

**Step 1: Funding Opportunity Announcement (FOA)**
Grant agency puts out an FOA (it's the government, did you think there wasn't an acronym?) that specifies what the grant is for, how much you could receive, who is eligible, timeline, and how to apply.  Federal FOA's are almost humorously specific, like this "[Small Grants Program for Leading English Discussion Groups in Sapporo, Japan](https://www.grants.gov/web/grants/search-grants.html)" 

**Step 2: Application and Award**
Agency seeking funding writes up their grant application (which usually is about the size of a graduate thesis) including a need statement, program narrative, proposed budget (or a budget narrative), discussion of impact, blood type, head shots, and $20 worth of pennies from 1960. Some of those are a joke but grant applicants have to be really careful what they include in their applications, using specific language, as the submissions are filtered through an [automated](https://www.nsf.gov/pubs/policydocs/pappg18_1/pappg_2.jsp#IIC2) check initally. 

**Step 3: Administration**
Remember that grant budget you put in your application?  Now you have to stick to it. The grant will specify how much money is allotted for each component: personnel, services, materials, etc.  Grant awardees are required to not only stick to these budgets, but also report weekly to the funding agency. 

For multi-year grants, awardees are oftentimes required to re-apply (with the understanding that they will still receive the grant).  Funding agencies may adjust the amount of funds given to awardees at this time.

#### Direct Services
Some non-profits receive funding for their programs through direct services given to members.  For example, a non-profit that offers Medication Assisted Treatment may submit insurance claims for participants who have received services through the non-profit.  Most insurance companies will have set money amounts for each procedure reported on each claim.


## Getting from A to B
### The Tech Boost

Back in the day, I was the data manager for two federal grants that were implemented by a state government.  Our grant provided direct services for citizens, so a portion of the grant was earmarked for those costs.  We were required to spend a certain percentage of grant funds on direct services.  

The program I worked for required a 60/40 split when it came to direct services.  This means we had to spend at least 60% of our allotted money on paying for medical services for our participants.  The grant also required that we report very specific data elements from those services. 

For the sake of easy math, let's say the program was granted $1,000,000.

In [63]:
#hide-input
breakdown = pd.DataFrame({'Resource':['Program Administrator','Data Manager', 'Case Manager','Program Specialist','Program Coordinator', 'Administrative', 'Direct Services'],
                         'Budget': [95000, 85000, 45000,75000,75000,20000,600000,]
                         })
breakdown["Program"] = "Program"

budget = alt.Chart(breakdown).mark_bar().encode(
    x=alt.X('Budget', title='Program Budget Breakdown'),
    y=alt.Y('Program', axis=alt.Axis(labels=False, title ='')),
    color=alt.Color('Resource', legend=None),
    tooltip=['Resource', 'Budget']
).configure_view(continuousHeight=600, continuousWidth=800)

budget

alt.Chart(...)

The grant requires that we find individuals in need of services, so we'll need a Program Coordinator.  It also requires a Program Specialist to engage with the organizations that provide the service and receive data reports from those organizations. This person should understand what data is required, how it comes in, will probably need to enter that data, and coordinate with service providers and participants regularly. The grant requires a Data Manager for bi-annual reporting of Minimum Data Elements (MDEs) identified and verified by the funding agency and ad-hoc program evaluation. The funding agency also requires regular reporting of progress and a specific adminstration of the grant, thus requiring a Program Administrator.  Because the staff for the program will need materials and space, there is an annual administrative budget.  This fits exactly into the $1,000,000 allotted. 

In [10]:
data = pd.read_csv("data/oes_all_x_tech.csv", encoding='utf-8')
jobs = pd.DataFrame(data=data)

In [14]:
pivoted = pd.DataFrame(data=jobs, columns = ["Industry", "Technologists", "total_emp"])
pivoted = pd.melt(pivoted, id_vars=['Industry'], value_vars = ['Technologists', 'total_emp'])
pivoted.head()

,Industry,variable,value
0,"Agriculture, Forestry, Fishing and Hunting",Technologists,290
1,Mining,Technologists,12630
2,Utilities,Technologists,19480
3,Construction,Technologists,18100
4,Manufacturing,Technologists,307140


In [19]:
convert_pivot = {'Industry': str, 
                'variable': str, 
                'value': int, 
               }
types = pivoted.astype(convert_pivot)


In [20]:
print(types.dtypes)

Industry    object
variable    object
value        int64
dtype: object


In [24]:
ratios = alt.Chart(types).mark_bar().encode(
    x='variable',
    y='value',
    color='Industry',
    tooltip=['Industry', 'value']
    )
ratios

alt.Chart(...)

In [30]:
alt.Chart(jobs).transform_joinaggregate(
    TotalEmp='sum(total_emp)',
).transform_calculate(
    PercentOfEmployed="(datum.total_emp-datum.Technologists)/datum.TotalEmp"
).mark_bar().encode(
    alt.X('PercentOfEmployed:Q', axis=alt.Axis(format='.0%')),
    y='Industry:N',
    tooltip=['total_emp'])

alt.Chart(...)

In [210]:
#convert objects

jobs["Employment_percent"] = pd.to_numeric(jobs["Employment_percent"], errors='coerce', downcast='float')
jobs = jobs.dropna(subset=['Employment_percent'])
jobs["Employment_percent"] = jobs["Employment_percent"].astype(float)

jobs["Employees"] = pd.to_numeric(jobs["Employees"], errors='coerce')
jobs = jobs.dropna(subset=['Employees'])
jobs["Employees"] = jobs["Employees"].astype(int)

In [12]:
print(jobs.dtypes)

Industry               object
Technologists           int64
total_emp               int64
Hourly mean wage      float64
annual_salary_tech      int64
annual_salary_all       int64
dtype: object


In [11]:
convert_dict = {'Industry': str, 
                'Technologists': int, 
                'total_emp': int, 
                'Hourly_mean_wage': float,
                'annual_salary_all': int,
                'annual_salary_tech': int,
               }
jobs = jobs.astype(convert_dict)

KeyError: 'Only a column name can be used for the key in a dtype mappings argument.'

In [ ]:
hierarchy = jobs["Industry"].str.split(' -', n=1, expand=True)
hierarchy.columns = ['industry_type', 'industry_sub_type']
jobs = join

In [217]:
# total employees for each industry and occupation
jobs['all_industry'] = jobs.groupby('Occupation')['Employees'].transform('max')
jobs['all_occupation'] = jobs.groupby('Industry')['Employees'].transform('max')
jobs['all_employed'] = jobs['Employees'].max()
jobs

,Industry,Occupation,Employees,Employment_percent,Hourly_mean_wage,Annual_mean_wage,10_ntile_annual,25_ntile_annual,50_ntile_annual,75_ntile_annual,90_ntile_annual,all_industry,all_occupation,all_employed
0,All,Computer and Mathematical Occupations,4552880,0.4,45.08,93760,45320,62850,88340,119550,152010,4552880,146875480,146875480
1,Private Industry,Computer and Mathematical Occupations,4107760,0.4,45.88,95430,45780,63920,90120,121860,154650,4552880,125009370,146875480
2,Retail,Computer and Mathematical Occupations,49940,7.5,38.74,80590,34020,47070,71280,106610,143320,4552880,15822440,146875480
3,Information,Computer and Mathematical Occupations,588500,1.5,48.99,101900,48070,67860,97140,131900,161570,4552880,2831080,146875480
4,Finance and Insurance,Computer and Mathematical Occupations,438650,1.5,47.63,99070,52050,70080,95330,123130,153580,4552880,5972610,146875480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Specialty hospitals - Private,Technologists,2220,5.1,41.91,87180,48220,63470,84580,108310,130450,4552880,232210,146875480
134,Social Assistance,Technologists,11050,3.9,28.67,59630,33040,40990,54680,73200,96190,4552880,3970540,146875480
135,Grantmaking and Giving Services,Technologists,4610,4.3,39.68,82520,43250,56010,76590,102260,132030,4552880,143660,146875480
136,Social Advocacy Organizations,Technologists,4470,5.1,36.41,75740,37110,50260,68500,97040,126580,4552880,222590,146875480


In [222]:
hierarchy = jobs["Industry"].str.split(' -', n=1, expand=True)
hierarchy.columns = ['industry_type', 'industry_sub_type']
jobs = jobs.join(hierarchy)

In [231]:
jobs.head()

,Industry,Occupation,Employees,Employment_percent,Hourly_mean_wage,Annual_mean_wage,10_ntile_annual,25_ntile_annual,50_ntile_annual,75_ntile_annual,90_ntile_annual,all_industry,all_occupation,all_employed,industry_type,industry_sub_type
0,All,Computer and Mathematical Occupations,4552880,0.4,45.08,93760,45320,62850,88340,119550,152010,4552880,146875480,146875480,All,None
1,Private Industry,Computer and Mathematical Occupations,4107760,0.4,45.88,95430,45780,63920,90120,121860,154650,4552880,125009370,146875480,Private Industry,None
2,Retail,Computer and Mathematical Occupations,49940,7.5,38.74,80590,34020,47070,71280,106610,143320,4552880,15822440,146875480,Retail,None
3,Information,Computer and Mathematical Occupations,588500,1.5,48.99,101900,48070,67860,97140,131900,161570,4552880,2831080,146875480,Information,None
4,Finance and Insurance,Computer and Mathematical Occupations,438650,1.5,47.63,99070,52050,70080,95330,123130,153580,4552880,5972610,146875480,Finance and Insurance,None


In [78]:
indx_jobs = jobs.set_index('Occupation')

ttl_employees = indx_jobs[lambda d: d.Industry == "All"].Employees
ttl_employees
#indx_jobs.Employees

Occupation
Computer and Mathematical Occupations      4552880
All                                      146875480
Database Administrators                     125460
Developers                                 1754750
Technologists                              4552880
Name: Employees, dtype: int64

In [81]:
indx_jobs.Employees / ttl_employees

Occupation
All              1.000000
All              0.851125
All              0.107727
All              0.019275
All              0.040664
                   ...   
Technologists    0.000488
Technologists    0.002427
Technologists    0.001013
Technologists    0.000982
Technologists    0.057970
Name: Employees, Length: 136, dtype: float64

In [188]:
from siuba import mutate, transmute


(jobs
>> group_by(_.Occupation)
>> mutate(pct_emp_job = _.Employees[_.Industry != 'All'].sum()/_..max()) 
#>> transmute(pct_employed = _.Employees/_.all_industry)
)

,Industry,Occupation,Employees,Employment_percent,Hourly_mean_wage,Annual_mean_wage,10_ntile_annual,25_ntile_annual,50_ntile_annual,75_ntile_annual,90_ntile_annual,total_emp_ind,all_industry,all_occupation,pct_emp_job
0,All,Computer and Mathematical Occupations,4552880,0.4,45.08,93760,45320,62850,88340,119550,152010,NaN,4552880,146875480,1.000000
1,Private Industry,Computer and Mathematical Occupations,4107760,0.4,45.88,95430,45780,63920,90120,121860,154650,NaN,4552880,125009370,0.902233
2,Retail,Computer and Mathematical Occupations,49940,7.5,38.74,80590,34020,47070,71280,106610,143320,NaN,4552880,15822440,0.010969
3,Information,Computer and Mathematical Occupations,588500,1.5,48.99,101900,48070,67860,97140,131900,161570,NaN,4552880,2831080,0.129259
4,Finance and Insurance,Computer and Mathematical Occupations,438650,1.5,47.63,99070,52050,70080,95330,123130,153580,NaN,4552880,5972610,0.096346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Specialty hospitals - Private,Technologists,2220,5.1,41.91,87180,48220,63470,84580,108310,130450,NaN,4552880,232210,0.000488
134,Social Assistance,Technologists,11050,3.9,28.67,59630,33040,40990,54680,73200,96190,NaN,4552880,3970540,0.002427
135,Grantmaking and Giving Services,Technologists,4610,4.3,39.68,82520,43250,56010,76590,102260,132030,NaN,4552880,143660,0.001013
136,Social Advocacy Organizations,Technologists,4470,5.1,36.41,75740,37110,50260,68500,97040,126580,NaN,4552880,222590,0.000982


In [226]:
upper_tiers = ['Hospitals', 'Real Estate', 'Social Advocacy Organizations', 'Social Assistance', 'Grantmaking and Giving Services', 'Educational Services', 'Finance and Insurance', 'Retail', 'Scientific Research', 'Higher Ed', 'Information', 'Social Assistance']

In [229]:
top_industry = (jobs 
                >>group_by(_.industry_type) 
                >>summarize(tech_emp = _.Employees[_.Occupation == 'Technologists'].sum())
                >>arrange(_.tech_emp)
               )
top_industry[top_industry["industry_type"].isin(upper_tiers)]


,industry_type,tech_emp
12,Real Estate,0
15,Social Advocacy Organizations,4470
6,Grantmaking and Giving Services,4610
16,Social Assistance,11050
13,Retail,49940
8,Hospitals,83660
14,Scientific Research,86560
7,Higher Ed,116770
2,Educational Services,231480
4,Finance and Insurance,438650


In [249]:
all_tech = jobs["Employees"].max()
top_industry["t_employed"] = all_tech
top_industry

,industry_type,tech_emp,t_employed
12,Real Estate,0,146875480
11,Psychiatric and Substance Abuse Hospitals,1490,146875480
17,Specialty hospitals,3480,146875480
15,Social Advocacy Organizations,4470,146875480
6,Grantmaking and Giving Services,4610,146875480
16,Social Assistance,11050,146875480
1,Ambulatory Health Services,47620,146875480
13,Retail,49940,146875480
5,General Medical and Surgical Hospitals,78690,146875480
8,Hospitals,83660,146875480


In [114]:
pvt_employed

,Industry,Occupation,Employees,Employment_percent,Hourly_mean_wage,Annual_mean_wage,10_ntile_annual,25_ntile_annual,50_ntile_annual,75_ntile_annual,90_ntile_annual,total_emp_ind,all_industry,all_occupation
29,Private Industry,All,125009370,0.1,25.2,52410,21160,26200,37970,61990,100380,NaN,146875480,125009370


In [201]:
pct_pvt = (pvt_employed["Employees"]/t_employed)*100
pct_pub = ((t_employed-pvt_employed["Employees"])/t_employed)*100
pct_pub

29    14.887516
Name: Employees, dtype: float64

In [255]:
df = pd.DataFrame(data = [pct_pvt, pct_pub], columns = {29: 30})
df

,29
Employees,85.112484
Employees,14.887516


In [253]:
df.columns = ["pct_employed"]

In [254]:
df

,pct_employed
Employees,85.112484
Employees,14.887516


85% of all employed persons in the US work in the private industry. 